In [1]:
import pandas as pd
import numpy as np
import pyodbc

In [4]:
df = pd.read_csv('ecommerce_40000.csv')

In [5]:
df['OrderDate'] = pd.to_datetime(df['OrderDate'])  
df['Year'] = df['OrderDate'].dt.year  
df['Quarter'] = df['OrderDate'].dt.to_period('Q')  
df[['Year', 'Quarter']]


,Year,Quarter
0,2024,2024Q2
1,2024,2024Q4
2,2022,2022Q3
3,2024,2024Q2
4,2023,2023Q1
...,...,...
39995,2022,2022Q4
39996,2024,2024Q4
39997,2024,2024Q4
39998,2024,2024Q3


In [6]:
df.dtypes

OrderID                      int64
CustomerID                   int64
OrderDate           datetime64[ns]
ProductID                    int64
ProductCategory             object
ProductName                 object
Quantity                     int64
PricePerUnit               float64
Discount                   float64
PaymentMethod               object
CustomerLocation            object
DeliveryTimeDays             int64
DeliveryStatus              object
SellerID                     int64
Rating                     float64
TotalPrice                 float64
Year                         int32
Quarter              period[Q-DEC]
dtype: object

In [7]:
quarter = df.groupby(['Year','Quarter']).size().to_frame()
quarterly = quarter.sort_values(by=['Year', 'Quarter'], ascending = False)
quarterly

0
Year Quarter      
2024 2024Q4   3319
     2024Q3   3290
     2024Q2   3283
     2024Q1   3377
2023 2023Q4   3238
     2023Q3   3362
     2023Q2   3326
     2023Q1   3267
2022 2022Q4   3420
     2022Q3   3487
     2022Q2   3369
     2022Q1   3262

In [ ]:
# seller performance

In [9]:
df.groupby('SellerID').agg(
    avg_rating = ('Rating','mean'),
    TotalOrders = ('OrderID','count')
).sort_values(by = ['avg_rating'],ascending = False).head(5).round(2)

,avg_rating,TotalOrders
SellerID,,
211,3.16,359
269,3.14,375
226,3.11,387
258,3.11,394
288,3.10,374


In [ ]:
df['Status'] =df['DeliveryStatus'].isin(['Returned','Cancelled'])

In [10]:
df.columns

Index(['OrderID', 'CustomerID', 'OrderDate', 'ProductID', 'ProductCategory',
       'ProductName', 'Quantity', 'PricePerUnit', 'Discount', 'PaymentMethod',
       'CustomerLocation', 'DeliveryTimeDays', 'DeliveryStatus', 'SellerID',
       'Rating', 'TotalPrice', 'Year', 'Quarter'],
      dtype='object')

In [17]:
# Step 1: Map DeliveryStatus to numeric values
# Assuming 'Returned' and 'Cancelled' are to be counted as 1, others as 0
df['DeliveryStatusNumeric'] = df['DeliveryStatus'].map(lambda x: 1 if x in ['Returned', 'Cancelled'] else 0)

# Step 2: Group and aggregate
category = df.groupby('ProductCategory').agg(
    totalorders=('OrderID', 'count'),
    returncancelcount=('DeliveryStatusNumeric', 'sum')
)

# Step 3: Calculate return/cancel rate
category['R/C rate %'] = (category['returncancelcount'] / category['totalorders']) * 100

# Step 4: Sort and round
category = category.sort_values(by='R/C rate %', ascending=False).round(2)

# Display result
category


,totalorders,returncancelcount,R/C rate %
ProductCategory,,,
Toys,6625,3373,50.91
Electronics,6572,3330,50.67
Clothing,6751,3380,50.07
Home & Kitchen,6634,3301,49.76
Books,6765,3360,49.67
Beauty,6653,3275,49.23


In [18]:
df['DeliveryStatus'].value_counts().to_frame()

,count
DeliveryStatus,
Returned,10097
Delivered,10013
Pending,9968
Cancelled,9922


In [19]:
df.columns

Index(['OrderID', 'CustomerID', 'OrderDate', 'ProductID', 'ProductCategory',
       'ProductName', 'Quantity', 'PricePerUnit', 'Discount', 'PaymentMethod',
       'CustomerLocation', 'DeliveryTimeDays', 'DeliveryStatus', 'SellerID',
       'Rating', 'TotalPrice', 'Year', 'Quarter', 'DeliveryStatusNumeric'],
      dtype='object')

In [ ]:
pd.options.display.float_format = '(:,2f)'.format

In [20]:
revenue = df.groupby('ProductName').agg(
    TotalRevenue =('TotalPrice','sum'),
    totalQTY =('Quantity', 'sum')
)

top_products = revenue.sort_values(by='TotalRevenue',
                                   ascending = False)
top_products

,TotalRevenue,totalQTY
ProductName,,
Item D,8.623427e+06,40202
Item A,8.619519e+06,40311
Item E,8.543836e+06,39452
Item B,8.402288e+06,39459
Item C,8.364209e+06,39616


In [25]:
region_analysis = df.groupby('CustomerLocation').agg(
    Totalorders = ('OrderID','count'),
    TotalRevenue = ('TotalPrice','sum')
)
region_analysis = region_analysis.sort_values(by='Totalorders',
                                  ascending = False)
region_analysis

,Totalorders,TotalRevenue
CustomerLocation,,
Central,8048,8.583032e+06
East,8046,8.547505e+06
West,8029,8.573505e+06
North,7967,8.421707e+06
South,7910,8.427530e+06


In [28]:
category_delivery_time = df.groupby('ProductCategory').agg(
    AvgDeliverytime = ('DeliveryTimeDays','mean'),
    TotalOrders = ('TotalPrice','sum')
)
delivery_time = category_delivery_time.sort_values(by='AvgDeliverytime',
                                  ascending = False)
delivery_time

,AvgDeliverytime,TotalOrders
ProductCategory,,
Home & Kitchen,7.559090,6.992920e+06
Electronics,7.521150,7.130742e+06
Books,7.490613,7.243186e+06
Beauty,7.480685,6.997797e+06
Clothing,7.478744,7.111567e+06
Toys,7.406642,7.077066e+06


In [32]:
cus_orders = df.groupby('CustomerID').agg(
    TotalOrders = ('OrderID','count')
)

def segment_customer(order_count):
    if order_count >= 20:
         return "High"
    elif (order_count >=10 and order_count < 20):
         return "Medium"
    else:
         return "Low"

cus_orders['CustomerSegment'] = [segment_customer(x) for x in cus_orders['TotalOrders']]

segment_distribution = cus_orders['CustomerSegment'].value_counts().reset_index()
segment_distribution

,CustomerSegment,count
0,Medium,2137
1,Low,1844
2,High,19


In [33]:
category_dis = df.groupby('ProductCategory').agg(
    AVG_Discount = ('Discount', 'mean')
)

category_dis['AVG Discount %'] = (category_dis['AVG_Discount'] * 100)

category_dis = category_dis.sort_values(by='AVG Discount %',
                                        ascending = False)
category_dis

,AVG_Discount,AVG Discount %
ProductCategory,,
Clothing,0.151979,15.197897
Toys,0.150919,15.091925
Beauty,0.150328,15.032767
Home & Kitchen,0.150216,15.021556
Books,0.149914,14.991426
Electronics,0.148857,14.885727


In [34]:
Top_City = df.groupby('CustomerLocation').agg(
    total_orders = ('TotalPrice','sum')
)
Top_City = Top_City.sort_values(by = 'total_orders',
                                ascending = False)
Top_City

,total_orders
CustomerLocation,
Central,8.583032e+06
West,8.573505e+06
East,8.547505e+06
South,8.427530e+06
North,8.421707e+06


In [36]:
Repeat_cus = df.groupby('CustomerID').agg(
    Total_orders = ('CustomerID','count'),
    Total_revenue = ('TotalPrice','sum')
)

Repeat_cus = Repeat_cus.sort_values(by = 'Total_orders', ascending = False)

Repeat_cus.head(5)

,Total_orders,Total_revenue
CustomerID,,
4081,23,27092.9906
4201,22,23994.3966
4747,22,20458.0306
1018,22,20101.6046
1386,21,17321.1376


In [37]:
Sellers = df.groupby('SellerID').agg(
    Total_revenue = ('TotalPrice','sum'),
    AVG_revenue = ('TotalPrice','mean')
)
Sellers = Sellers.sort_values(by= 'Total_revenue', ascending = False)

Sellers.head(5)

,Total_revenue,AVG_revenue
SellerID,,
225,518947.1171,1198.492187
235,500242.8975,1121.620846
289,494221.8849,1136.142264
209,480991.2946,1134.413431
229,479393.2519,1120.077691


In [38]:
AVG_order = df.groupby('PaymentMethod').agg(

    Revenue = ('TotalPrice', 'sum'),
    AVG = ('TotalPrice', 'mean'),
    Total_orders = ('PaymentMethod','count')

)

AVG_order = AVG_order.sort_values(by = 'Total_orders', ascending = False)
AVG_order

,Revenue,AVG,Total_orders
PaymentMethod,,,
Cash on Delivery,8.653863e+06,1066.271935,8116
Net Banking,8.518186e+06,1062.117946,8020
Debit Card,8.456876e+06,1062.288150,7961
UPI,8.494752e+06,1067.582313,7957
Credit Card,8.429601e+06,1060.860980,7946


In [40]:
R_C = df.groupby('DeliveryStatus').agg(
     Revenue = ('TotalPrice','sum'),
    AVG = ('TotalPrice','mean'),
    Total_orders = ('DeliveryStatus','count')

)

R_C = R_C.sort_values(by = 'Total_orders', ascending = False)

R_C

,Revenue,AVG,Total_orders
DeliveryStatus,,,
Returned,1.075086e+07,1064.757600,10097
Delivered,1.058850e+07,1057.475574,10013
Pending,1.064785e+07,1068.203266,9968
Cancelled,1.056607e+07,1064.913138,9922


In [41]:
High = df.groupby('CustomerID').agg(
    Max = ('TotalPrice','max')
)

High = High.sort_values(by = 'Max', ascending = False)

High.head(5)

,Max
CustomerID,
3067,4480.2000
4191,4458.3300
3429,4413.4794
1943,4400.0334
2724,4389.8904


In [43]:
Product_category = df.groupby('ProductCategory').agg(
    AVG = ('Rating','mean'),
    total_rating = ('Rating', 'sum')
)

Product_category = Product_category.sort_values(by = 'AVG')

Product_category.head(5)

,AVG,total_rating
ProductCategory,,
Electronics,2.994324,19678.7
Home & Kitchen,2.995734,19873.7
Toys,3.000815,19880.4
Clothing,3.003111,20274.0
Books,3.006977,20342.2


In [46]:
revenue_by_category_year = df.groupby(['ProductCategory', 'Year'])['TotalPrice'].sum().reset_index()
pivot = revenue_by_category_year.pivot(index='ProductCategory', columns='Year', values='TotalPrice').fillna(0)

years = sorted(pivot.columns)

for i in range(1, len(years)):
    prev, curr = years[i-1], years[i]
    pivot[f'YoY_Growth_(prev)_(curr) (%)'] = ((pivot[curr] + pivot[prev]) / pivot[prev].replace(0, pd.NA)) * 100

print(pivot.reset_index().head(5))

Year ProductCategory          2022          2023          2024  \
0             Beauty  2.329629e+06  2.320030e+06  2.348137e+06   
1              Books  2.397491e+06  2.429950e+06  2.415745e+06   
2           Clothing  2.438307e+06  2.351372e+06  2.321888e+06   
3        Electronics  2.372034e+06  2.372961e+06  2.385747e+06   
4     Home & Kitchen  2.450911e+06  2.346206e+06  2.195802e+06   

Year  YoY_Growth_(prev)_(curr) (%)  
0                       201.211494  
1                       199.415411  
2                       198.746128  
3                       200.538841  
4                       193.589485  


In [48]:
Product_price = df.groupby('ProductCategory').agg(
    Totalprice = ('PricePerUnit','mean')
)

def segment_product(order_price):
    if order_price >=100:
        return "High"
    elif (order_price >= 100 and order_price < 300):
        return "Medium"
    else:
        return "Low"

Product_price['ProductSegment'] = [segment_product(x) for x in Product_price['Totalprice']]

segment_distribution = Product_price['ProductSegment'].reset_index()
segment_distribution

,ProductCategory,ProductSegment
0,Beauty,High
1,Books,High
2,Clothing,High
3,Electronics,High
4,Home & Kitchen,High
5,Toys,High


In [52]:
q75 = df['TotalPrice'].quantile(0.75)
q25 = df['TotalPrice'].quantile(0.25)
df['ValueSegment'] = df['TotalPrice'].apply(lambda x: 'High' if x > q75 else 'Low' if x < q25 else 'Mid')

returns = df[df['DeliveryStatus'].isin(['Returned', 'Cancelled'])]
total_counts = df['ValueSegment'].value_counts()
return_counts = returns['ValueSegment'].value_counts()
return_rate = (return_counts / total_counts * 100).fillna(0)
return_rate

ValueSegment
Mid     49.805
High    50.360
Low     50.220
Name: count, dtype: float64

In [53]:
def delivery_band(x):
    return 'Fast' if x <= 3 else 'Moderate' if x <=7 else 'Slow'
df['DeliveryPerformanceBand'] =df['DeliveryTimeDays'].apply(delivery_band)
band_rating = df.groupby('DeliveryPerformanceBand')['Rating'].mean().reset_index()
band_rating

,DeliveryPerformanceBand,Rating
0,Fast,2.993299
1,Moderate,3.014287
2,Slow,2.999183


In [55]:
df['Quarter'] =pd.to_datetime(df['OrderDate']).dt.quarter
df['IsPeakSeason'] = df['Quarter'] == 4
peak = df.groupby('IsPeakSeason').agg(
    TotalOrders=('OrderID', 'count'),
    TotalRevenue=('TotalPrice', 'sum')
).reset_index()
peak

,IsPeakSeason,TotalOrders,TotalRevenue
0,False,30023,3.192116e+07
1,True,9977,1.063212e+07


In [56]:
seller_rev =df.groupby('SellerID')['TotalPrice'].sum().reset_index()
quantiles = seller_rev['TotalPrice'].quantile([0.33, 0.66])
seller_rev['Tier'] = pd.cut(
    seller_rev['TotalPrice'],
    bins=[-float('inf'), quantiles[0.33], quantiles[0.66], float('inf')],
    labels=['Low', 'Mid', 'High']
)
completion = df[df['DeliveryStatus'] == 'Delivered'].groupby('SellerID')['OrderID'].count() / df.groupby('SellerID')['OrderID'].count()
seller_rev['CompletionRate'] = seller_rev['SellerID'].map(completion)
tier_completion = seller_rev.groupby('Tier')['CompletionRate'].mean().reset_index()

tier_completion

C:\Users\Yamuna\AppData\Local\Temp\ipykernel_19484\2615824196.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tier_completion = seller_rev.groupby('Tier')['CompletionRate'].mean().reset_index()


,Tier,CompletionRate
0,Low,0.249641
1,Mid,0.246395
2,High,0.254514


In [58]:
customer_orders = df.groupby('CustomerID')['OrderID'].count().reset_index(name='OrderCount')

def assign_band(count):
    if count == 1:
        return 'Rare'
    elif 2 <= count <= 4;
        return 'Occasional'
    else:
        return 'Frequent'

customer_orders['CustomerFrequencyBand'] = customer_orders['OrderCount'].apply(assign_band)

df = df.merge(customer_orders[('CustomerID', 'CustomerFrequencyBand')], on='CustomerID', how='left')

revenue_by_band = (
    df.groupby('CustomerFrequencyBand')['TotalPrice']
      .sum()
      .reset_index()
      .sort_values(by='TotalPrice', ascending=False)
)

revenue_by_band

SyntaxError: invalid syntax (2320548176.py, line 6)

In [59]:
seller_stats = df.groupby('SellerID').agg(
    Avg_Discoungt=('Discount', 'mean'),
    Total_Revenue=('TotalPrice', 'sum')
).reset_index()

seller_stats

,SellerID,Avg_Discoungt,Total_Revenue
0,200,0.154375,444060.1880
1,201,0.151741,400401.3994
2,202,0.144613,420221.6476
3,203,0.147758,424983.3410
4,204,0.146839,429932.7203
...,...,...,...
95,295,0.159034,396169.6688
96,296,0.145956,456009.6102
97,297,0.150146,432404.6101
98,298,0.151051,416100.2762
